In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# Helper functions

In [31]:
def print_long_string(string, indent=0, max_words_per_line=10):
    """
    :param string: str
    :param indent: int
    :param max_words_per_line: int
    :return: None
    """
    words = [" "*indent]
    for i, word in enumerate(string.split()):
        words.append(word)
        if (i+1) % max_words_per_line == 0:
            words.append("\n" + " "*indent)
    print(" ".join(words))
    return None


def print_col_desc(df, table_name, col_desc_df):
    """
    To print out the information of columns of df. This information is taken from col_desc_df
    :param df: dataframe
    :param table_name: str
    :param col_desc_table:
    :return: None
    """
    nrows = df.shape[0]
    all_cols = df.columns.to_list()
    cat_cols = df.select_dtypes(["category", "object", "bool"]).columns.to_list()
    num_cols = [col for col in all_cols if col not in cat_cols]

    for i, col in enumerate(df.columns):
        mask = (col_desc_df["Table"] == table_name) & (col_desc_df["Row"] == col)
        print("Column Number:", i)
        print("Column Name:", col)
        print("Description:")

        if len(col_desc_df.loc[mask, :]) == 0:
            print(" " * 10 + "No Description, Maybe the column name does not match.")
        else:
            print_long_string(col_desc_df.loc[mask, "Description"].iloc[0], indent=10)
            print("Special:", col_desc_df.loc[mask, "Special"].iloc[0])

        print("Type:", df[col].dtype)
        n_null = np.sum(df[col].isnull())
        print("Number of rows:", nrows)
        print("Number of NULL(s):", n_null)
        print("Percent of NULL(s): %.2f%%" % (n_null / nrows * 100))

        nunique = df[col].nunique(dropna=False)
        print("Number of Unique Values:", nunique)
        if nunique <= 20:
            print_long_string(", ".join([str(s) for s in df[col].unique()]),
                                indent=28, max_words_per_line=5)
        else:
            print_long_string(", ".join([str(s) for s in df[col].unique()[:20]]) + " ...",
                                indent=28, max_words_per_line=5)

        if col in num_cols:
            print("Min:", df[col].min())
            print("Max:", df[col].max())
        else:
            print("Mode: ", df[col].mode().values)

        print("-" * 50 + "\n")
    return None


def change_dtypes(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        if (df[col].dtype == "object") and (df[col].nunique() < df.shape[0]):
            df[col] = df[col].astype("category")

        elif set(df[col].unique()) == set([0, 1]):
            df[col] = df[col].astype(bool)

        elif df[col].dtype == float:
            df[col] = df[col].astype(np.float32)

        elif df[col].dtype == int:
            df[col] = df[col].astype(np.int32)

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df

In [3]:
col_desc = pd.read_csv("data/download/HomeCredit_columns_description.csv", encoding="ISO-8859-1")

# `application_train.csv`

This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).

Static data for all applications. One row represents one loan in our data sample.

In [5]:
application_train = pd.read_csv("data/download/application_train.csv")
application_train = change_dtypes(application_train)
print("application_train.shape:", application_train.shape)
application_train.head()

Memory usage before changing types 300.13 MB
Memory usage after changing types 104.87 MB
application_train.shape: (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,True,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,False,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,False,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,False,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,False,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
application_train.shape

(307511, 122)

In [6]:
application_train.dtypes

SK_ID_CURR                       int32
TARGET                            bool
NAME_CONTRACT_TYPE            category
CODE_GENDER                   category
FLAG_OWN_CAR                  category
                                ...   
AMT_REQ_CREDIT_BUREAU_DAY      float32
AMT_REQ_CREDIT_BUREAU_WEEK     float32
AMT_REQ_CREDIT_BUREAU_MON      float32
AMT_REQ_CREDIT_BUREAU_QRT      float32
AMT_REQ_CREDIT_BUREAU_YEAR     float32
Length: 122, dtype: object

In [7]:
application_train["SK_ID_CURR"].is_unique

True

In [32]:
print_col_desc(application_train, "application_{train|test}.csv", col_desc)

Column Number: 0
Column Name: SK_ID_CURR
Description:
           ID of loan in our sample
Special: nan
Type: int32
Number of rows: 307511
Number of NULL(s): 0
Percent of NULL(s): 0.00%
Number of Unique Values: 307511
                             100002, 100003, 100004, 100006, 100007, 
                             100008, 100009, 100010, 100011, 100012, 
                             100014, 100015, 100016, 100017, 100018, 
                             100019, 100020, 100021, 100022, 100023 
                             ...
Min: 100002
Max: 456255
--------------------------------------------------

Column Number: 1
Column Name: TARGET
Description:
           Target variable (1 - client with payment difficulties: he/she had 
           late payment more than X days on at least one 
           of the first Y installments of the loan in our 
           sample, 0 - all other cases)
Special: nan
Type: bool
Number of rows: 307511
Number of NULL(s): 0
Percent of NULL(s): 0.00%
Number of Unique

                             False, True
Mode:  [False]
--------------------------------------------------

Column Number: 28
Column Name: OCCUPATION_TYPE
Description:
           What kind of occupation does the client have
Special: nan
Type: category
Number of rows: 307511
Number of NULL(s): 96391
Percent of NULL(s): 31.35%
Number of Unique Values: 19
                             Laborers, Core staff, Accountants, Managers, 
                             nan, Drivers, Sales staff, Cleaning 
                             staff, Cooking staff, Private service 
                             staff, Medicine staff, Security staff, 
                             High skill tech staff, Waiters/barmen 
                             staff, Low-skill Laborers, Realty agents, 
                             Secretaries, IT staff, HR staff 
                            
Mode:  ['Laborers']
Categories (18, object): ['Accountants', 'Cleaning staff', 'Cooking staff', 'Core staff', ..., 'Sales staff', 'Secre

Min: 0.0
Max: 1.0
--------------------------------------------------

Column Number: 53
Column Name: LANDAREA_AVG
Description:
           Normalized information about building where the client lives, What is 
           average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment 
           size, common area, living area, age of building, number of 
           elevators, number of entrances, state of the building, number of 
           floor
Special: normalized
Type: float32
Number of rows: 307511
Number of NULL(s): 182590
Percent of NULL(s): 59.38%
Number of Unique Values: 3528
                             0.0369, 0.013, nan, 0.0135, 0.0931, 
                             0.1758, 0.0279, 0.0534, 0.0898, 0.2371, 
                             0.0861, 0.0498, 0.0265, 0.0872, 0.1569, 
                             0.038, 0.0888, 0.0566, 0.1404, 0.0142 
                             ...
Min: 0.0
Max: 1.0
--------------------------------------------------

Column Number: 54
Co

                             0.0144, 0.0497, nan, 0.0587, 0.1153, 
                             0.0018, 0.0121, 0.0, 0.0214, 0.001, 
                             0.019, 0.0057, 0.0143, 0.083, 0.0533, 
                             0.0388, 0.0029, 0.0006, 0.0097, 0.0131 
                             ...
Min: 0.0
Max: 1.0
--------------------------------------------------

Column Number: 63
Column Name: ELEVATORS_MODE
Description:
           Normalized information about building where the client lives, What is 
           average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment 
           size, common area, living area, age of building, number of 
           elevators, number of entrances, state of the building, number of 
           floor
Special: normalized
Type: float32
Number of rows: 307511
Number of NULL(s): 163891
Percent of NULL(s): 53.30%
Number of Unique Values: 27
                             0.0, 0.0806, nan, 0.1611, 0.4028, 
                             0

                             0.025, 0.0968, nan, 0.0833, 0.1489, 
                             0.3529, 0.0281, 0.0729, 0.0916, 0.1457, 
                             0.1447, 0.0167, 0.152, 0.0125, 0.3706, 
                             0.0937, 0.0958, 0.0822, 0.1322, 0.2207 
                             ...
Min: 0.0
Max: 1.0
--------------------------------------------------

Column Number: 73
Column Name: BASEMENTAREA_MEDI
Description:
           Normalized information about building where the client lives, What is 
           average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment 
           size, common area, living area, age of building, number of 
           elevators, number of entrances, state of the building, number of 
           floor
Special: normalized
Type: float32
Number of rows: 307511
Number of NULL(s): 179943
Percent of NULL(s): 58.52%
Number of Unique Values: 3773
                             0.0369, 0.0529, nan, 0.0973, 0.1335, 
                  

Min: 0.0
Max: 1.0
--------------------------------------------------

Column Number: 83
Column Name: LIVINGAREA_MEDI
Description:
           Normalized information about building where the client lives, What is 
           average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment 
           size, common area, living area, age of building, number of 
           elevators, number of entrances, state of the building, number of 
           floor
Special: normalized
Type: float32
Number of rows: 307511
Number of NULL(s): 154350
Percent of NULL(s): 50.19%
Number of Unique Values: 5282
                             0.0193, 0.0558, nan, 0.0792, 0.1422, 
                             0.3842, 0.0295, 0.063, 0.0889, 0.151, 
                             0.1239, 0.0682, 0.0096, 0.1437, 0.0152, 
                             0.262, 0.0613, 0.0648, 0.0592, 0.0938 
                             ...
Min: 0.0
Max: 1.0
--------------------------------------------------

Column Number: 84


                             2.0, 1.0, 0.0, 4.0, 8.0, 
                             10.0, nan, 7.0, 3.0, 6.0, 
                             5.0, 12.0, 9.0, 13.0, 11.0, 
                             14.0, 21.0, 15.0, 22.0, 16.0 
                             ...
Min: 0.0
Max: 344.0
--------------------------------------------------

Column Number: 94
Column Name: DEF_60_CNT_SOCIAL_CIRCLE
Description:
           How many observation of client's social surroundings defaulted on 60 
           (days past due) DPD
Special: nan
Type: float32
Number of rows: 307511
Number of NULL(s): 1021
Percent of NULL(s): 0.33%
Number of Unique Values: 10
                             2.0, 0.0, 1.0, nan, 3.0, 
                             5.0, 4.0, 7.0, 24.0, 6.0 
                            
Min: 0.0
Max: 24.0
--------------------------------------------------

Column Number: 95
Column Name: DAYS_LAST_PHONE_CHANGE
Description:
           How many days before application did client change phone
Special: nan


In [39]:
application_test = pd.read_csv("data/download/application_test.csv")
application_test = change_dtypes(application_test)

Memory usage before changing types 47.18 MB
Memory usage after changing types 18.19 MB


In [41]:
nrows_train_test = application_train.shape[0] + application_test.shape[0]
nrows_train_test

356255

# `bureau.csv`

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).

For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

In [34]:
bureau = pd.read_csv("data/download/bureau.csv")
print("bureau.shape:", bureau.shape)
bureau.head()

bureau.shape: (1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [35]:
bureau["SK_ID_CURR"].is_unique

False

In [38]:
bureau["SK_ID_CURR"].nunique()

305811

In [43]:
# there are about 24% of applications do not have bureau data
bureau["SK_ID_CURR"].nunique() / nrows_train_test

0.8584047943186762

In [36]:
bureau["SK_ID_BUREAU"].is_unique

True

In [ ]:
print_col_desc(bureau, "bureau.csv", col_desc)

# Description of columns in table `bureau_balance.csv`

Monthly balances of previous credits in Credit Bureau.

This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

In [ ]:
bureau_balance = pd.read_csv("data/download/bureau_balance.csv")
print("bureau_balance.shape:", bureau_balance.shape)
bureau_balance.head()

In [ ]:
bureau_balance["SK_ID_BUREAU"].is_unique

In [ ]:
print_col_desc(bureau_balance, "bureau_balance.csv", col_desc)

# Description of columns in table `previous_application.csv`

All previous applications for Home Credit loans of clients who have loans in our sample.

There is one row for each previous application related to loans in our data sample.

In [ ]:
previous_application = pd.read_csv("data/download/previous_application.csv")
print("previous_application.shape:", previous_application.shape)
previous_application.head()

In [ ]:
previous_application["SK_ID_PREV"].is_unique

In [ ]:
previous_application["SK_ID_CURR"].is_unique

In [ ]:
print_col_desc(previous_application, "previous_application.csv", col_desc)

# Description of columns in table `POS_CASH_balance.csv`

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.

This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

In [ ]:
POS_CASH_balance = pd.read_csv("data/download/POS_CASH_balance.csv")
print("POS_CASH_balance.shape:", POS_CASH_balance.shape)
POS_CASH_balance.head()

In [ ]:
[id_col for id_col in POS_CASH_balance.columns.values if id_col.startswith("SK_ID")]

In [ ]:
POS_CASH_balance["SK_ID_PREV"].is_unique

In [ ]:
POS_CASH_balance["SK_ID_CURR"].is_unique

In [ ]:
POS_CASH_balance.groupby(by=["SK_ID_PREV"])["SK_ID_CURR"].count().head()

In [ ]:
tmp = POS_CASH_balance.groupby(by=["SK_ID_PREV"])["SK_ID_CURR"].nunique()
print(tmp.head())

# so each "SK_ID_PREV" is corresponding to a unique "SK_ID_CURR"
tmp[tmp > 1]

In [ ]:
print_col_desc(POS_CASH_balance, "POS_CASH_balance.csv", col_desc)

# Description of columns in table `credit_card_balance.csv`

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.

This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

In [ ]:
credit_card_balance = pd.read_csv("data/download/credit_card_balance.csv")
print("credit_card_balance.shape:", credit_card_balance.shape)
credit_card_balance.head()

In [ ]:
credit_card_balance["SK_ID_PREV"].is_unique

In [ ]:
credit_card_balance["SK_ID_CURR"].is_unique

In [ ]:
credit_card_balance.groupby(by=["SK_ID_PREV"])["SK_ID_CURR"].count().head()

In [ ]:
tmp = credit_card_balance.groupby(by=["SK_ID_PREV"])["SK_ID_CURR"].nunique()
print(tmp.head())

# so each "SK_ID_PREV" is corresponding to a unique "SK_ID_CURR"
tmp[tmp > 1]

In [ ]:
print_col_desc(credit_card_balance, "credit_card_balance.csv", col_desc)

# Description of columns in table  `installments_payments.csv`

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.

There is a) one row for every payment that was made plus b) one row each for missed payment.

One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

In [ ]:
installments_payments = pd.read_csv("data/download/installments_payments.csv")
print("installments_payments.shape:", installments_payments.shape)
installments_payments.head()

In [ ]:
installments_payments["SK_ID_PREV"].is_unique

In [ ]:
installments_payments["SK_ID_CURR"].is_unique

In [ ]:
installments_payments.groupby(by=["SK_ID_PREV"])["SK_ID_CURR"].count().head()

In [ ]:
tmp = installments_payments.groupby(by=["SK_ID_PREV"])["SK_ID_CURR"].nunique()
print(tmp.head())

# so each "SK_ID_PREV" is corresponding to a unique "SK_ID_CURR"
tmp[tmp > 1]

In [ ]:
print_col_desc(installments_payments, "installments_payments.csv", col_desc)